In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define diffusion schedule
class DiffusionScheduler:
    def __init__(self, timesteps=1000, beta_start=0.0001, beta_end=0.02):
        self.timesteps = timesteps
        self.beta = np.linspace(beta_start, beta_end, timesteps, dtype=np.float32)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = np.cumprod(self.alpha)

    def get_noise_level(self, t):
        return self.beta[t], self.alpha_bar[t]

diffusion_scheduler = DiffusionScheduler()

# Forward diffusion function
def forward_diffusion(x_0, t, noise_level=1.0):
    noise = torch.randn_like(x_0) * noise_level
    beta_t, alpha_bar_t = diffusion_scheduler.get_noise_level(t)
    x_t = torch.sqrt(torch.tensor(alpha_bar_t)) * x_0 + torch.sqrt(torch.tensor(1 - alpha_bar_t)) * noise
    return x_t, noise

# Define U-Net Model for noise prediction
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.enc1 = nn.Sequential(nn.Conv1d(9, 32, 3, padding=1), nn.ReLU(), nn.MaxPool1d(2))
        self.enc2 = nn.Sequential(nn.Conv1d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool1d(2))
        self.enc3 = nn.Sequential(nn.Conv1d(64, 128, 3, padding=1), nn.ReLU())

        self.dec2 = nn.Sequential(nn.Upsample(scale_factor=2, mode='nearest'), nn.Conv1d(128, 64, 3, padding=1), nn.ReLU())
        self.dec1 = nn.Sequential(nn.Upsample(scale_factor=2, mode='nearest'), nn.Conv1d(64, 32, 3, padding=1), nn.ReLU())

        self.final = nn.Conv1d(32, 9, 1, padding=0)

    def forward(self, x):
        enc1_out = self.enc1(x)
        enc2_out = self.enc2(enc1_out)
        bottleneck = self.enc3(enc2_out)

        dec2_out = self.dec2(bottleneck)
        dec1_out = self.dec1(dec2_out)

        return self.final(dec1_out)

# Create synthetic dataset
def create_dataset(num_samples=1000, seq_length=256, num_features=9):
    return torch.randn((num_samples, num_features, seq_length))

dataset = create_dataset()

# Model Initialization
unet_model = UNet()
optimizer = optim.Adam(unet_model.parameters(), lr=0.0001)
criterion = nn.MSELoss()

# Training Loop
EPOCHS = 10
BATCH_SIZE = 32
TIMESTEPS = 1000
for epoch in range(EPOCHS):
    for i in range(0, dataset.shape[0], BATCH_SIZE):
        batch = dataset[i:i+BATCH_SIZE]
        t = np.random.randint(0, TIMESTEPS)
        noisy_x, noise = forward_diffusion(batch, t)

        optimizer.zero_grad()
        predicted_noise = unet_model(noisy_x)
        loss = criterion(predicted_noise, noise)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {loss.item():.4f}")

# Sampling function (Reverse Process)
def sample(num_samples=1, timesteps=1000):
    x_t = torch.randn((num_samples, 9, 256))  # Random noise
    for t in reversed(range(timesteps)):
        predicted_noise = unet_model(x_t)
        beta_t, alpha_bar_t = diffusion_scheduler.get_noise_level(t)
        x_t = (x_t - torch.sqrt(torch.tensor(1 - alpha_bar_t)) * predicted_noise) / torch.sqrt(torch.tensor(alpha_bar_t))
        x_t += torch.sqrt(torch.tensor(beta_t)) * torch.randn_like(x_t)  # Add small noise
    return x_t

# Generate new samples
generated_samples = sample(num_samples=5)
print("Generated Samples Shape:", generated_samples.shape)

# Anomaly Detection Function
def anomaly_score(real, recon):
    return torch.mean(torch.abs(real - recon), dim=1)



Epoch 1/10, Loss: 1.0031
Epoch 2/10, Loss: 0.9839
Epoch 3/10, Loss: 1.0007
Epoch 4/10, Loss: 0.9522
Epoch 5/10, Loss: 0.9386
Epoch 6/10, Loss: 0.9194
Epoch 7/10, Loss: 0.9130
Epoch 8/10, Loss: 0.8777
Epoch 9/10, Loss: 1.0318
Epoch 10/10, Loss: 0.8703
Generated Samples Shape: torch.Size([5, 9, 256])


In [2]:
# Generate Test Data
num_test_samples = 100
test_data = torch.randn((num_test_samples, 9, 256))
noisy_test, _ = forward_diffusion(test_data, np.random.randint(0, TIMESTEPS))

In [ ]:
# Reconstruct using trained model
reconstructed_test = sample(num_samples=num_test_samples)

# Compute anomaly scores
anomaly_scores = anomaly_score(test_data, reconstructed_test)

# Identify anomalies (threshold can be tuned)
anomaly_threshold = torch.mean(anomaly_scores) + 2 * torch.std(anomaly_scores)
anomalies = anomaly_scores > anomaly_threshold

print(f"Number of detected anomalies: {torch.sum(anomalies).item()}")